In [1]:
import tensorflow as tf
from tensorflow.keras import models , layers

In [2]:
IMAGE_SIZE=256
BATCH_SIZE=8
CHANNELS=3

In [3]:
dataset =tf.keras.preprocessing.image_dataset_from_directory(
    "Pepper_Bell",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 2475 files belonging to 2 classes.


In [4]:
class_names = dataset.class_names
class_names

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy']

In [5]:
def get_dataset_partitions_tf(ds,train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):

  ds_size =len(ds)

  if shuffle:
    ds=ds.shuffle(shuffle_size)

  train_size = int(train_split*ds_size)
  val_size = int(val_split*ds_size)

  train_ds = ds.take(train_size)

  val_ds=ds.skip(train_size).take(val_size)
  test_ds=ds.skip(train_size).skip(val_size)


  return train_ds ,val_ds, test_ds

In [6]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


In [8]:
len(train_ds)

248

In [9]:
len(val_ds)

31

In [10]:
len(test_ds)

31

In [11]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [12]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)

])

In [13]:
data_augmentation = tf.keras.Sequential([
     layers.RandomFlip("horizontal_and_vertical"),
     layers.RandomRotation(0.2),
])

In [14]:
input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes=2
model1=models.Sequential([
    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Flatten(),

    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])
model1.build(input_shape=input_shape)

c:\Users\rekha\OneDrive\Desktop\Plant Disease Detection\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [15]:
model1.summary

<bound method Model.summary of <Sequential name=sequential_2, built=True>>

In [16]:
model1.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [18]:
checkpoint = ModelCheckpoint('./pepper_bell_model.keras',
    verbose=1,
    monitor='val_accuracy',
    save_best_only=True,
    mode='auto'
)

In [19]:
EPOCHS=15
history=model1.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

Epoch 1/15
248/248 ━━━━━━━━━━━━━━━━━━━━ 0s 615ms/step - accuracy: 0.6403 - loss: 0.6368
Epoch 1: val_accuracy improved from -inf to 0.91129, saving model to ./pepper_bell_model.keras
248/248 ━━━━━━━━━━━━━━━━━━━━ 183s 680ms/step - accuracy: 0.6407 - loss: 0.6364 - val_accuracy: 0.9113 - val_loss: 0.2827
Epoch 2/15
248/248 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - accuracy: 0.9276 - loss: 0.3272
Epoch 2: val_accuracy improved from 0.91129 to 0.97984, saving model to ./pepper_bell_model.keras
248/248 ━━━━━━━━━━━━━━━━━━━━ 126s 508ms/step - accuracy: 0.9276 - loss: 0.3269 - val_accuracy: 0.9798 - val_loss: 0.0636
Epoch 3/15
248/248 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step - accuracy: 0.9738 - loss: 0.0832
Epoch 3: val_accuracy did not improve from 0.97984
248/248 ━━━━━━━━━━━━━━━━━━━━ 106s 426ms/step - accuracy: 0.9738 - loss: 0.0833 - val_accuracy: 0.9435 - val_loss: 0.1372
Epoch 4/15
248/248 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 0.9819 - loss: 0.1119
Epoch 4: val_accuracy did not improve fro

In [19]:
scores=model1.evaluate(test_ds)

31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 1.0000 - loss: 0.0010


In [20]:
history.params

{'verbose': 1, 'epochs': 15, 'steps': 248}

In [21]:
history.history.keys()

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])

In [22]:
history.history['accuracy']

[0.6018190979957581,
 0.8029307723045349,
 0.9327943325042725,
 0.9742293953895569,
 0.9813036918640137,
 0.9873673319816589,
 0.9838302135467529,
 0.993431031703949,
 0.9883779883384705,
 0.9893885850906372,
 0.9853461384773254,
 0.989893913269043,
 0.9914097785949707,
 0.9893885850906372,
 0.989893913269043]

In [25]:
import numpy as np

In [26]:
def predict(model,img):
  img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
  img_array = tf.expand_dims(img_array,0)# Create a batch

  predictions = model.predict(img_array)

  predicted_class = class_names[np.argmax(predictions[0])]
  confidence = round(100* (np.max(predictions[0])),2)
  return predicted_class, confidence

In [29]:
for images, labels in test_ds.take(1):
  for i in range(9):
    predicted_class, confidence=predict(model1,images[i].numpy())
    print(predicted_class)
    print(confidence)
    actual_class = class_names[labels[i]]
    print(actual_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Pepper__bell___healthy
99.95
Pepper__bell___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Pepper__bell___Bacterial_spot
100.0
Pepper__bell___Bacterial_spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Pepper__bell___healthy
99.97
Pepper__bell___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Pepper__bell___healthy
99.94
Pepper__bell___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Pepper__bell___healthy
99.96
Pepper__bell___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Pepper__bell___healthy
99.95
Pepper__bell___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Pepper__bell___healthy
99.97
Pepper__bell___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Pepper__bell___Bacterial_spot
97.39
Pepper__bell___Bacterial_spot


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 8 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/